In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# This is the code for the Linear Model
from statsmodels.formula.api import ols

from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt
from sklearn.model_selection import train_test_split
from pydataset import data

import warnings
warnings.filterwarnings('ignore')

#### Load the tips dataset.

<li>Create a column named tip_percentage. This should be the tip amount divided by the total bill.
<li>Create a column named price_per_person. This should be the total bill divided by the party size.
    
    

In [9]:
tips= data('tips')
tips['tip_percentage']= tips.tip/tips.total_bill
tips['price_per_person']= tips.total_bill/tips.size
#removing non numeric columns for the purpose of this exercise
tips=tips.drop(columns=['sex', 'smoker', 'day', 'time'])
tips.head()

,total_bill,tip,size,tip_percentage,price_per_person
1,16.99,1.01,2,0.059447,0.008704
2,10.34,1.66,3,0.160542,0.005297
3,21.01,3.50,3,0.166587,0.010763
4,23.68,3.31,2,0.139780,0.012131
5,24.59,3.61,4,0.146808,0.012597


<li>Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?<br>
  <i>  I think the most important feature for predicting tip amount is the time of the meal and the tip percentage would be sex. </i>

<li>Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [3]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing

In [6]:
def split(df, stratify_by=""):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    return train, validate, test DataFrames.
    '''
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)
    return train, validate, test
train, validate, test = split(tips, stratify_by='tip')

In [7]:
target = "tip"

# split train into X (dataframe, drop target) & y (series, keep target only)
X_train = train.drop(columns=[target])
y_train = train[target]

# split validate into X (dataframe, drop target) & y (series, keep target only)
X_validate = validate.drop(columns=[target])
y_validate = validate[target]

# split test into X (dataframe, drop target) & y (series, keep target only)
X_test = test.drop(columns=[target])
y_test = test[target]

X_train.shape, X_validate.shape, X_test.shape

((136, 4), (59, 4), (49, 4))

In [10]:
# Scale the data
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler
scaler.fit(X_train)

# Use the scaler to transform train, validate, test
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

<li>Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [11]:

#Number of features
k = 2

# Let's start with Select K Best
# Make the thing
kbest = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression, k=2)

# fit the thing
kbest.fit(X_train, y_train)

# use the thing, 
# get_support() produces an array of booleans, so we can filter out the column names that matter the most
kbest_features = X_train.columns[kbest.get_support()].tolist()

print("KBest's 2 best features are", kbest_features)

KBest's 2 best features are ['total_bill', 'price_per_person']


In [13]:
# Make the thing(s)
lm = sklearn.linear_model.LinearRegression()
# # of reafures to use
rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=2)

# Fit the thing
rfe.fit(X_train, y_train)

# use the thing
rfe_columns = X_train.columns[rfe.support_].tolist()

print("RFE's 2 best features are", rfe_columns)


RFE's 2 best features are ['size', 'tip_percentage']


<li>Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

#### Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [35]:
def kbest_features(X, y, k):
    '''This function takes in a X-Train, Y Train and number of features then returns names of the top number of features based in the K Best Object.'''
    # make the object
    kbest = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression, k=k)
    # fit the object
    kbest.fit(X, y)
    # use the object (.get_support() is that array of booleans to filter the list of column names)
    return X.columns[kbest.get_support()].tolist()

In [17]:
kbest_features(X_train, y_train, 2)

['total_bill', 'price_per_person']

#### Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [37]:
def features_rankings(X_train, rfe):
    """
    Takes in a dataframe and a fit RFE object in order to output the rank of all features
    """
    # rfe here is reference rfe from cell 15
    var_ranks = rfe.ranking_
    var_names = X_train.columns.tolist()
    ranks = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    ranks = ranks.sort_values(by="Rank", ascending=True)
    return ranks
def rfe_features(X, y, r):
    ''' This function takes in a X-Train, Y Train and number of features then returns names of the top number of features based on the RFE object.'''
    # make the object
    lm = sklearn.linear_model.LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)
    # Fit the object
    rfe.fit(X, y)
    # use the object
    features_to_use = X.columns[rfe.support_].tolist()
    # we need to send show_feature_rankings a trained/fit RFE object
    all_rankings = features_rankings(X, rfe)
    return features_to_use, all_rankings

In [21]:
rfe_features(X_train, y_train, 2)

(['size', 'tip_percentage'],
                 Var  Rank
 1              size     1
 2    tip_percentage     1
 0        total_bill     2
 3  price_per_person     3)

#### Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [24]:
swiss = data('swiss')


In [25]:
# Split the data
train, validate, test = split(swiss, stratify_by="Fertility")

# Setup X and y
X_train = train.drop(columns='Fertility')
y_train = train.Fertility

X_validate = validate.drop(columns='Fertility')
y_validate = validate.Fertility

X_test = test.drop(columns='Fertility')
y_test = test.Fertility

In [26]:
# Scale the data
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler
scaler.fit(X_train)

# Use the scaler to transform train, validate, test
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)


# Turn everything into a dataframe
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_validate_scaled = pd.DataFrame(X_validate_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_train.columns)

In [31]:
print("The top", k,"kbest features are", kbest_features(X_train, y_train, 2))

The top 2 kbest features are ['Examination', 'Catholic']


In [39]:
print("The top", k,"RFE features are", rfe_features(X_train, y_train, 2))

The top 2 RFE features are (['Examination', 'Infant.Mortality'],                 Var  Rank
1       Examination     1
4  Infant.Mortality     1
0       Agriculture     2
2         Education     3
3          Catholic     4)
